In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split

primary_df=pd.read_csv('/content/drive/MyDrive/signate/従業員の離職予測/train.csv')
ans_df=primary_df.loc[:,'Attrition']
primary_df.drop('Attrition',axis=1,inplace=True)

In [6]:
def reshape_df(df):
    #idは意味を持たないため削除
    df.drop('id',axis=1,inplace=True)

    #print(train_df["StandardHours"].value_counts())
    #80    1200
    #Name: StandardHours, dtype: int64
    df.drop('StandardHours',axis=1,inplace=True)

    #print(train_df["Over18"].value_counts())
    #Y    1200
    #Name: Over18, dtype: int64
    df.drop('Over18',axis=1,inplace=True)

        #適当に
    df.drop('EmployeeNumber',axis=1,inplace=True)
    df.drop('JobLevel',axis=1,inplace=True)
    df.drop('MonthlyIncome',axis=1,inplace=True)
    df.drop('YearsAtCompany',axis=1,inplace=True)
    df.drop('YearsSinceLastPromotion',axis=1,inplace=True)
    df.drop('WorkLifeBalance',axis=1,inplace=True)
    df.drop('NumCompaniesWorked',axis=1,inplace=True)

    #https://deepage.net/features/pandas-replace.html
    df=df.replace({
        #BusinessTravel
        'Non-Travel':0,
        'Travel_Rarely':1,
        'Travel_Frequently':2,
        #Gender
        'Male':0,
        'Female':1,
        #OverTime
        'No':0,
        'Yes':1,
    })

    #https://note.nkmk.me/python-pandas-get-dummies/
    df=pd.get_dummies(df, columns=["Department", "EducationField","JobRole","MaritalStatus"],sparse=True)
    return df

import scipy.stats
def preprocess(df):
    df=reshape_df(df)
    return scipy.stats.zscore(df.values,ddof=1)

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split

primary_df=pd.read_csv('/content/drive/MyDrive/signate/従業員の離職予測/train.csv')
ans_df=primary_df.loc[:,'Attrition']
primary_df.drop('Attrition',axis=1,inplace=True)

x_train,x_test,y_train,y_test=train_test_split(primary_df,ans_df, test_size=0.1)

x_train=x_train.loc[y_train==0,:]

x_train=preprocess(x_train)
x_test=preprocess(x_test)

In [13]:
from sklearn.svm import OneClassSVM
#https://scikit-learn.org/stable/modules/generated/sklearn.svm.OneClassSVM.html
clf = OneClassSVM(nu=0.0001, kernel='rbf', gamma='auto')
clf.fit(x_train)
y_pred=clf.predict(x_test)
y_pred[y_pred==1]=0
y_pred[y_pred==-1]=1

In [14]:
from sklearn.metrics import confusion_matrix
print((y_test[:]==y_pred[:]).sum()/len(y_test))
print(confusion_matrix(y_test,y_pred, labels=[1, 0]))

0.6333333333333333
[[ 5 25]
 [19 71]]


In [ ]:
import numpy as np
test_df=pd.read_csv('/content/drive/MyDrive/signate/従業員の離職予測/test.csv')
x_test=preprocess(test_df)
pred=clf.predict(x_test)
pred[pred==1]=0
pred[pred==-1]=1

data=np.array([test_df["id"].values,pred],dtype='int16')
data=pd.DataFrame(data.T)
data.to_csv('/content/drive/MyDrive/signate/従業員の離職予測/out.csv',index=False,header=False)